In [16]:
import geopandas as gpd
import vaex
import json

In [7]:
# Load Vaex DataFrame with points
df = vaex.open('data/trips/trips.hdf5')

df.info()

column,type,unit,description,expression
Trip ID,str,,,
Taxi ID,str,,,
Trip Start Timestamp,str,,,
Trip End Timestamp,str,,,
Trip Seconds,float64,,,
Trip Miles,float64,,,
Pickup Census Tract,float64,,,
Dropoff Census Tract,float64,,,
Pickup Community Area,float64,,,
Dropoff Community Area,float64,,,


In [18]:
# Load Vaex DataFrame
df = vaex.open('data/trips/trips.hdf5')

# Convert the column from string to GeoJSON format
def parse_json(x):
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return None

df['Pickup Centroid Location'] = df['Pickup Centroid Location'].apply(parse_json)

# Verify the updated column format
print(df['Pickup Centroid Location'].dtype)

# Save the updated Vaex DataFrame
#df.export_hdf5('path/to/updated_vaexfile.hdf5')


object


In [19]:
df.info()

column,type,unit,description,expression
Trip ID,str,,,
Taxi ID,str,,,
Trip Start Timestamp,str,,,
Trip End Timestamp,str,,,
Trip Seconds,float64,,,
Trip Miles,float64,,,
Pickup Census Tract,float64,,,
Dropoff Census Tract,float64,,,
Pickup Community Area,float64,,,
Dropoff Community Area,float64,,,


In [12]:
# Load GeoDataFrame with hexagonal shapes from GeoJSON
gdf = gpd.read_file('data/hexagon_files/hexagon_res7.geojson')

# Load Vaex DataFrame with points
df = vaex.open('data/trips/trips.hdf5')

# Create a spatial index for faster spatial queries
gdf.sindex

# Define a function to check if a point lies within a hexagon
def is_point_inside_hexagon(point, hexagon):
    return point.any().within(hexagon)

# Create a new column in the Vaex DataFrame to store the join result
df['hexagon_tripstart_res7'] = '-1'  # Initialize with a default value indicating no match

# Iterate over each row in the GeoDataFrame
for index, hexagon in gdf.iterrows():
    # print(hexagon["geometry"])
    # Check if any point in the Vaex DataFrame lies within the hexagon
    points_in_hexagon = df[df['Pickup Centroid Location'].apply(lambda point: is_point_inside_hexagon(point, hexagon))]
    if len(points_in_hexagon) > 0:
        # Set the hexagon ID for the matching points in the Vaex DataFrame
        df.loc[points_in_hexagon.index, 'hexagon_tripstart_res7'] = index

# # Save the updated Vaex DataFrame with the hexagon IDs
# df.export_hdf5('path/to/joined_vaexfile.hdf5')


AttributeError: 'str' object has no attribute 'any'